# Building an AWS<sup>®</sup> ML Pipeline with SageWorks (Classification)

<div style="padding: 20px">
<img width="1000" alt="sageworks_pipeline" src="https://github.com/SuperCowPowers/sageworks/assets/4806709/47cc5739-971c-48c3-9ef6-fd8370e3ec57"></div>

This notebook uses the SageWorks Science Workbench to quickly build an AWS® Machine Learning Pipeline with the AQSolDB public dataset. This dataset aggregates aqueous solubility data for a large set of compounds.

We're going to set up a full AWS Machine Learning Pipeline from start to finish. Since the SageWorks Classes encapsulate, organize, and manage sets of AWS® Services, setting up our ML pipeline will be straight forward.

SageWorks also provides visibility into AWS services for every step of the process so we know exactly what we've got and how to use it.
<br><br>

## Data
Wine Dataset: A classic dataset used in pattern recognition, machine learning, and data mining, the Wine dataset comprises 178 wine samples sourced from three different cultivars in Italy. The dataset features 13 physico-chemical attributes for each wine sample, providing a multi-dimensional feature space ideal for classification tasks. The aim is to correctly classify the wine samples into one of the three cultivars based on these chemical constituents. This dataset is widely employed for testing and benchmarking classification algorithms and is notable for its well-balanced distribution among classes. It serves as a straightforward, real-world example for classification tasks in machine learning.

**Main Reference:**
Forster, P. (1991). Machine Learning of Natural Language and Ontology (Technical Report DAI-TR-261). Department of Artificial Intelligence, University of Edinburgh.

**Important Note:** We've made a small change to the wine dataset to have string based target column called 'wine_class' with string labels instead of integer.

**Download Data** 

<a href="https://github.com/SuperCowPowers/sageworks/blob/main/data/wine_dataset.csv" download> Modified wine_dataset.csv</a>

## SageWorks
SageWorks is a medium granularity framework that manages and aggregates AWS® Services into classes and concepts. When you use SageWorks you think about DataSources, FeatureSets, Models, and Endpoints. Underneath the hood those classes handle all the details around updating and

## Notebook
This notebook uses the SageWorks Science Workbench to quickly build an AWS® Machine Learning Pipeline.

We're going to set up a full AWS Machine Learning Pipeline from start to finish. Since the SageWorks Classes encapsulate, organize, and manage sets of AWS® Services, setting up our ML pipeline will be straight forward.

SageWorks also provides visibility into AWS services for every step of the process so we know exactly what we've got and how to use it.
<br><br>

® Amazon Web Services, AWS, the Powered by AWS logo, are trademarks of Amazon.com, Inc. or its affiliates.

In [1]:
# SageWorks has verbose log messages so set to warning
import sageworks
import logging
logging.getLogger("sageworks").setLevel(logging.WARNING)

2024-02-15 06:31:58 (sageworks_logging.py:142) INFO SageWorks Logging Setup Complete...


In [2]:
# Note: If you want to use local data just use a file path
from sageworks.api.data_source import DataSource
s3_path = "s3://sageworks-public-data/common/wine_dataset.csv"
data_source = DataSource(s3_path, 'wine_data')

<div style="float: right; padding: 20px"><img src="images/aws_dashboard_aqsol.png" width=600px"></div>

# So what just happened?
Okay, so it was just a few lines of code but SageWorks did the following for you:
   
- Transformed the CSV to a **Parquet** formatted dataset and stored it in AWS S3
- Created an AWS Data Catalog database/table with the columns names/types
- Athena Queries can now be done directly on this data in AWS Athena Console

The new 'DataSource' will show up in AWS and of course the SageWorks AWS Dashboard. Anyone can see the data, get information on it, use AWS® Athena to query it, and of course use it as part of their analysis pipelines.

<div style="float: right; padding: 20px"><img src="images/athena_query_aqsol.png" width=600px"></div>

# Visibility and Easy to Use AWS Athena Queries
Since SageWorks manages a broad range of AWS Services it means that you get visibility into exactly what data you have in AWS. It also means nice perks like hitting the 'Query' link in the Dashboard Web Interface and getting a direct Athena console on your dataset. With AWS Athena you can use typical SQL statements to inspect and investigate your data.
    
**But that's not all!**
    
SageWorks also provides API to directly query DataSources and FeatureSets right from the API, so lets do that now.

In [3]:
# Athena queries are easy
data_source.query('SELECT * from wine_data limit 5')

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280_od315_of_diluted_wines,proline,wine_class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,TypeA
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,TypeA
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,TypeA
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,TypeA
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,TypeA


# Labels can be strings
We can see in the dataframe above that our target column has **strings** in it. You do not need to convert these to integers, just use the transformation classes and a LabelEncoder will be used internally for training and prediction/inference.

# The AWS ML Pipeline Awaits
Okay, so in a few lines of code we created a 'DataSource' (which is simply a set of orchestrated AWS Services) but now we'll go through the construction of the rest of our Machine Learning pipeline.

<div style="padding: 20px">
<img width="1000" alt="sageworks_pipeline" src="https://github.com/SuperCowPowers/sageworks/assets/4806709/47cc5739-971c-48c3-9ef6-fd8370e3ec57"></div>

## ML Pipeline
- DataSource **(done)**
- FeatureSet
- Model
- Endpoint (serves models)

# Create a FeatureSet
**Note:** Normally this is where you'd do a deep dive on the data/features, look at data quality metrics, redudant features and engineer new features. For the purposes of this notebook we're simply going to take the given 13 physico-chemical attributes for each wine sample.

In [4]:
data_source.column_details()

{'alcohol': 'double',
 'malic_acid': 'double',
 'ash': 'double',
 'alcalinity_of_ash': 'double',
 'magnesium': 'double',
 'total_phenols': 'double',
 'flavanoids': 'double',
 'nonflavanoid_phenols': 'double',
 'proanthocyanins': 'double',
 'color_intensity': 'double',
 'hue': 'double',
 'od280_od315_of_diluted_wines': 'double',
 'proline': 'double',
 'wine_class': 'string'}

In [5]:
help(data_source.to_features)

Help on method to_features in module sageworks.api.data_source:

to_features(name: str = None, tags: list = None, target_column: str = None, id_column: str = None, event_time_column: str = None, one_hot_encode: bool = True) -> sageworks.api.feature_set.FeatureSet method of sageworks.api.data_source.DataSource instance
    Convert the DataSource to a FeatureSet
    
    Args:
        name (str): Set the name for feature set (must be lowercase). If not specified, a name will be generated
        tags (list): Set the tags for the feature set. If not specified tags will be generated.
        target_column (str): Set the target column for the feature set. (Optional)
        id_column (str): Set the id column for the feature set. If not specified will be generated.
        event_time_column (str): Set the event time for the feature set. If not specified will be generated.
        one_hot_encode (bool): Whether to one-hot encode categorical fields (default: True)
    
    Returns:
        Fea

# Creating the FeatureSet (takes at least 15 minutes)

# Why does creating a FeatureSet take a long time?
Great question, between row 'ingestion' and waiting for the offline store to finish populating itself it does take a **long time**. SageWorks is simply invoking the AWS Service APIs and those APIs are taking a while to do their thing.

The good news is that SageWorks can monitor and query the status of the object and let you know when things are ready.

In [ ]:
data_source.to_features("wine_features", target_column="wine_class", tags=["wine", "classification", "uci"])

# New FeatureSet shows up in Dashboard
Now we see our new feature set automatically pop up in our dashboard. FeatureSet creation involves the most complex set of AWS Services:
- New Entry in AWS Feature Store
- Specific Type and Field Requirements are handled
- Plus all the AWS Services associated with DataSources (see above)

The new 'FeatureSet' will show up in AWS and of course the SageWorks AWS Dashboard. Anyone can see the feature set, get information on it, use AWS® Athena to query it, and of course use it as part of their analysis pipelines.

<div style="padding: 20px"><img src="images/dashboard_aqsol_features.png" width=1000px"></div>
    
**Important:** All inputs are stored to track provenance on your data as it goes through the pipeline. We can see the last field in the FeatureSet shows the input DataSource.

# Publishing our Model
**Note:** Normally this is where you'd do a deep dive on the feature set. For the purposes of this notebook we're simply going to take the features given to us and make a reference model that can track our baseline model performance for other to improve upon. :)

In [10]:
from sageworks.api.feature_set import FeatureSet
from sageworks.api.model import Model, ModelType

fs = FeatureSet("wine_features")
help(fs.to_model)

Help on method to_model in module sageworks.api.feature_set:

to_model(model_type: sageworks.core.artifacts.model_core.ModelType, target_column: str, name: str = None, tags: list = None, description: str = None, feature_list: list = None) -> sageworks.api.model.Model method of sageworks.api.feature_set.FeatureSet instance
    Create a Model from the FeatureSet
    
    Args:
        model_type (ModelType): The type of model to create (See sageworks.model.ModelType)
        target_column (str): The target column for the model (use None for unsupervised model)
        name (str): Set the name for the model. If not specified, a name will be generated
        tags (list): Set the tags for the model.  If not specified tags will be generated.
        description (str): Set the description for the model. If not specified a description is generated.
        feature_list (list): Set the feature list for the model. If not specified a feature list is generated.
    
    Returns:
        Model: Th

In [8]:
fs.column_names()

['write_time',
 'api_invocation_time',
 'is_deleted',
 'alcohol',
 'malic_acid',
 'ash',
 'alcalinity_of_ash',
 'magnesium',
 'total_phenols',
 'flavanoids',
 'nonflavanoid_phenols',
 'proanthocyanins',
 'color_intensity',
 'hue',
 'od280_od315_of_diluted_wines',
 'proline',
 'wine_class',
 'id',
 'event_time']

In [ ]:
tags = ["wine", "classification", "public"]
fs.to_model(ModelType.CLASSIFIER, target_column="wine_class", name="wine-classification", 
            tags=tags, description="Wine Classification Model")

# Deploying an AWS Endpoint
Okay now that are model has been published we can deploy an AWS Endpoint to serve inference requests for that model. Deploying an Endpoint allows a large set of servies/APIs to use our model in production.

In [ ]:
model = Model("wine-classification"
model.to_endpoint("wine-classification-end", tags=["wine", "classification"])

# Model Inference from the Endpoint
AWS Endpoints will bundle up a model as a service that responds to HTTP requests. The typical way to use an endpoint is to send a POST request with your features in CSV format. SageWorks provides a nice DataFrame based interface that takes care of many details for you.

In [11]:
# Get the Endpoint
from sageworks.api.endpoint import Endpoint
my_endpoint = Endpoint('wine-classification-end')

# Model Provenance is locked into SageWorks
We can now look at the model, see what FeatureSet was used to train it and even better see exactly which ROWS in that training set where used to create the model. We can make a query that returns the ROWS that were not used for training.

In [13]:
table = fs.get_training_view_table()
test_df = fs.query(f"select * from {table} where training=0")
test_df.head()

,write_time,api_invocation_time,is_deleted,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280_od315_of_diluted_wines,proline,wine_class,id,event_time,training
0,2024-02-15 13:39:31.289000+00:00,2024-02-15 13:33:54+00:00,False,12.70,3.55,2.36,21.5,106.0,1.70,1.20,0.17,0.84,5.00,0.78,1.29,600.0,TypeC,133,2024-02-15T13:33:30.813Z,0
1,2024-02-15 13:39:31.289000+00:00,2024-02-15 13:33:54+00:00,False,12.00,0.92,2.00,19.0,86.0,2.42,2.26,0.30,1.43,2.50,1.38,3.12,278.0,TypeB,80,2024-02-15T13:33:30.813Z,0
2,2024-02-15 13:39:31.215000+00:00,2024-02-15 13:33:54+00:00,False,14.75,1.73,2.39,11.4,91.0,3.10,3.69,0.43,2.81,5.40,1.25,2.73,1150.0,TypeA,13,2024-02-15T13:33:30.813Z,0
3,2024-02-15 13:39:20.942000+00:00,2024-02-15 13:33:54+00:00,False,13.50,3.12,2.62,24.0,123.0,1.40,1.57,0.22,1.25,8.60,0.59,1.30,500.0,TypeC,150,2024-02-15T13:33:30.813Z,0
4,2024-02-15 13:39:31.242000+00:00,2024-02-15 13:33:54+00:00,False,12.36,3.83,2.38,21.0,88.0,2.30,0.92,0.50,1.04,7.65,0.56,1.58,520.0,TypeC,160,2024-02-15T13:33:30.813Z,0


In [14]:
# Okay now use the SageWorks Endpoint to make prediction on TEST data
prediction_df = my_endpoint.predict(test_df)
metrics = my_endpoint.classification_metrics("wine_class", prediction_df)
metrics

Processing...


,wine_class,precision,recall,fscore,roc_auc,support
2,TypeA,1.0,1.0,1.0,1.0,11
1,TypeB,1.0,1.0,1.0,1.0,12
0,TypeC,1.0,1.0,1.0,1.0,14


# Follow Up on Predictions
Looking at the prediction plot above we can see that many predictions were close to the actual value but about 10 of the predictions were WAY off. So at this point we'd use SageWorks to investigate those predictions, map them back to our FeatureSet and DataSource and see if there were irregularities in the training data.

# Wrap up: Building an AWS<sup>®</sup> ML Pipeline with SageWorks

<div style="float: right; padding: 20px"><img width="450" src="https://user-images.githubusercontent.com/4806709/266844238-df2f1b90-9e6f-4dbb-9490-ad75545e630f.png"></div>



This notebook used the SageWorks Science Toolkit to quickly build an AWS® Machine Learning Pipeline with the AQSolDB public dataset. We built a full AWS Machine Learning Pipeline from start to finish. 

SageWorks made it easy:
- Visibility into AWS services for every step of the process.
- Managed the complexity of organizing the data and populating the AWS services.
- Provided an easy to use API to perform Transformations and inspect Artifacts.

Using SageWorks will minimizize the time and manpower needed to incorporate AWS ML into your organization. If your company would like to be a SageWorks Alpha Tester, contact us at [sageworks@supercowpowers.com](mailto:sageworks@supercowpowers.com).

<br><br><br><br>
<br><br><br><br>
<br><br><br><br>
<br><br><br><br>
<br><br><br><br>
<br><br><br><br>

# Helper Methods

In [ ]:
# Plotting defaults
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-deep')
#plt.style.use('seaborn-dark')
plt.rcParams['font.size'] = 12.0
plt.rcParams['figure.figsize'] = 14.0, 7.0